In [4]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv


load_dotenv()

api_key = os.getenv("API_KEY")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key = api_key
)


In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)

In [8]:
ai_msg.content

'Je adore programmer.'

In [9]:
llm.invoke("The first person to land on the moon was...").content

'The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.'

In [12]:
import chromadb 
chroma_client = chromadb.Client()

In [13]:
collection = chroma_client.create_collection(name="my_collection")


In [14]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


C:\Users\biswa\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [03:05<00:00, 448kiB/s]   


In [18]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [20]:
collection.delete(ids = ['id1','id2'])
collection.get()


{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [94]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/137734004324344518-staff-software-engineer-core-machine-learning-google-cloud?tag=ai-spotlight")
page_data = loader.load().pop().page_content
print(page_data)

Staff Software Engineer, Core Machine Learning, Google Cloud — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results5  jobs matchedStaff Software Engineer, Core Machine Learning, Google CloudSunnyvale, CA, USA; United States | Remote eligibleSenior Software Engineer, Machine Learning, Google Cloud ComputeKirkland, WA, USA; Seattle, WA, USA; +2 more; +1 moreAbuse Analyst, Trust and Safety, MessagingKirkland, WA, USA; Austin, TX, USAData Scientist, Extended Workforce SolutionsHyderabad, Telangana, IndiaSenior Software Engineer, Natural Language Processing, Google ResearchTel Aviv, Israel1‑5 of 5Follow Life at Google on More about usAbout usopen_in_newContact usopen_in_newPressopen_in_newRelated InformationInvestor

In [123]:
from langchain_core.prompts import PromptTemplate

extraction_prompt = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract a single job posting and return it in JSON format containing the 
        following keys: `role`, `experience`, `skills`, and `description`.
        skills value is not a dictionary, rather just contains the skills. 
        Only return a valid JSON object without wrapping it in a list and no examples required.
        ### VALID JSON (NO PREAMBLE):
    """
)


jobs = results.get("jobs_results", [])
for job in jobs: 
    extraction_chain = extraction_prompt | llm
    res = extraction_chain.invoke(input={job['description']})
    print(res.content)


```json
{
  "role": "Software Test Engineer",
  "experience": {
    "required": {
      "years": "at least 3 years",
      "description": "test automation (web/API)"
    },
    "preferred": {
      "years": "not specified",
      "description": "software development lifecycle, procedure design, and writing"
    }
  },
  "skills": [
    "Cypress",
    "Postman",
    "Git",
    "Azure DevOps",
    "C# .NET",
    "C++",
    "English (at least professional level)"
  ],
  "description": {
    "job_description": "At Philips Capsule, we are driven by the challenge of “Making the complex simple, and the simple insightful.” To achieve this, our team needs a new tester with a strong affinity for automation.",
    "responsibilities": [
      "Design test strategies to ensure the product meets functional and non-functional requirements.",
      "Conduct methodological reviews of specifications to ensure testability.",
      "Write detailed test procedures and implement manual and automated test pl

In [124]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer',
 'experience': 'Degree in Computer Science, Engineering, or a related field.',
 'skills': 'Proficient in Typescript, Strong problem-solving abilities, Excellent communication, Ability to work independently, Low Ego and Team spirit mindset, Comfortable with UI development (for example React, Angular, Vue.js), Kotlin experience is a plus',
 'description': 'As a Senior Software Engineer specializing in IDE extension development at Mistral AI, you will be at the forefront of innovation, enhancing developer productivity and integrating cutting-edge AI capabilities into their IDE. Your role will be pivotal in bridging the gap between AI research and practical application, creating tools that empower developers to achieve more with less effort.'}

In [125]:
type(json_res)

dict

In [89]:
import pandas as pd

df = pd.read_csv("./app/resource/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [126]:
job = json_res
job['skills']

'Proficient in Typescript, Strong problem-solving abilities, Excellent communication, Ability to work independently, Low Ego and Team spirit mindset, Comfortable with UI development (for example React, Angular, Vue.js), Kotlin experience is a plus'

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [127]:
job = json_res
job['skills']

'Proficient in Typescript, Strong problem-solving abilities, Excellent communication, Ability to work independently, Low Ego and Team spirit mindset, Comfortable with UI development (for example React, Angular, Vue.js), Kotlin experience is a plus'

In [128]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
print(links)

[[{'links': 'https://example.com/typescript-frontend-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}]]


In [68]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your AI Potential with AtliQ

Dear Hiring Manager,

I came across the job description for a Staff Software Engineer, Core Machine Learning, Google Cloud, and I was impressed by the requirements and responsibilities outlined. As a business development executive at AtliQ, an AI & Software Consulting company, I believe our expertise and capabilities align perfectly with your needs.

AtliQ has a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has a deep understanding of machine learning algorithms and tools, artificial intelligence, deep learning, and natural language processing, which are essential skills for this role.

I'd like to highlight a few examples of our work that demonstrate our capabilities:

* Our team has developed a robust machine learning pipeline using Python and TensorFlow, which can be viewed on our portfolio page: h

In [ ]:
from serpapi.google_search import GoogleSearch

params = {
    "engine": "google_jobs",
    "q": "Software Engineer",  # Job title or query
    "location": "Paris, France",  # Job location
    "api_key": "4c066377097b3f2defb418eb6ee27be11cf538a7ef62f7c0867df76ab6899602"  # Replace with your actual API key
}

search = GoogleSearch(params)
results = search.get_dict()

jobs = results.get("jobs_results", [])
for job in jobs:
    print(f"Title: {job['title']}")
    print(f"Company: {job['company_name']}")
    print(f"Location: {job['location']}")
    print(f"Location: {job['extensions']}")
    print("-" * 50)


Title: Software Test Engineer
Company: Philips
Location: Paris, France
--------------------------------------------------
Title: Lead Software Engineer
Company: Bain & Company
Location: Paris, France
--------------------------------------------------
Title: Senior Software Engineer, Full Stack - Paris
Company: Mistral AI
Location: Paris, France
--------------------------------------------------
Title: Staff Software Engineer
Company: GitGuardian
Location: Paris, France
--------------------------------------------------
Title: Software Test Engineer
Company: Philips
Location: Paris, France
--------------------------------------------------
Title: Software Engineer in Test - Onsite or Remote (FR, UK, DE, NL)
Company: Dataiku
Location: Anywhere
--------------------------------------------------
Title: Software Engineer - Developer Experience
Company: Datadog
Location: Paris, France
--------------------------------------------------
Title: Software Engineer - Algorithms
Company: Criteo
Loc

In [111]:
jobs['Software Test Engineer'].get(['skills'])

TypeError: list indices must be integers or slices, not str

In [116]:
jobs = results.get("jobs_results", [])
for job in jobs:
    print(f"Title: {job['title']}")
    print(f"Company: {job['company_name']}")
    print(f"Location: {job['location']}")
    print(f"Detected: {job['job_highlights']['items']}")
    print("-" * 50)

Title: Software Test Engineer
Company: Philips
Location: Paris, France


KeyError: 'job_highlights'

In [117]:
jobs_results = results["jobs_results"]

In [118]:
jobs_results

[{'title': 'Software Test Engineer',
  'company_name': 'Philips',
  'location': 'Paris, France',
  'via': 'Philips Careers',
  'share_link': 'https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=doqVo54uxJRGHVj3AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQoCMQwAUFzvC8Qps3itCC46q-B0oPuRK7Gt1KQ0Ae8L_G51eePrPouuv8nD3tgI7qQGJ46ZiRr0cJUJlLCFBMJwEYmFVsdkVvXgvWpxUQ0tBxfk5YVpktk_ZdI_o6bfWQsajbv9dnaV43o5pFxyVcgMA7asGzg35EBfs7cDy4kAAAA&shmds=v1_AQbUm97SjbKclLn561bjBtX4Yo3dvUqK5sMVNV9CqOmEJN9eRw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=doqVo54uxJRGHVj3AAAAAA%3D%3D',
  'thumbnail': 'https://serpapi.com/searches/67e85feca8dd4ed061071113/images/f6a6dd5068e40b5129269413efdd17d8bbe17eba11cc84cb76c5356404c3a554.png',
  'extensions': ['6 days ago', 'Full-time'],
  'detected_extensions': {'posted_at': '6 days ago',
   'schedule_type': 'Full-time'},
  'description': "Job Title\nSoftware Test Engineer\n\nJob Description\n\nAt Philips C

In [129]:
from serpapi.google_search import GoogleSearch

params = {
    "engine": "google_jobs",
    "q": "Software Engineer",  # Job title or query
    "location": "Paris, France",  # Job location
    "api_key": "4c066377097b3f2defb418eb6ee27be11cf538a7ef62f7c0867df76ab6899602"  # Replace with your actual API key
}

search = GoogleSearch(params)
results = search.get_dict()

jobs = results.get("jobs_results", [])
for job in jobs:
    print(f"Title: {job['title']}")
    print(f"Company: {job['company_name']}")
    print(f"Location: {job['location']}")
    print(f"Location: {job['extensions']}")
    print("-" * 50)


Title: Software Engineer - Full Stack
Company: Dext France
Location: Paris, France
Location: ['4 days ago', 'Full-time', 'No degree mentioned']
--------------------------------------------------
Title: Lead Software Engineer
Company: Bain & Company
Location: Paris, France
Location: ['10 days ago', 'Full-time']
--------------------------------------------------
Title: Software Test Engineer
Company: Philips
Location: Paris, France
Location: ['6 days ago', 'Full-time']
--------------------------------------------------
Title: Senior Software Engineer, Full Stack - Paris
Company: Mistral AI
Location: Paris, France
Location: ['27 days ago', 'Full-time', 'No degree mentioned']
--------------------------------------------------
Title: Staff Software Engineer
Company: GitGuardian
Location: Paris, France
Location: ['13 days ago', 'Full-time', 'No degree mentioned']
--------------------------------------------------
Title: Software Engineer - Algorithms
Company: Criteo
Location: Paris, France (